In [77]:
import os
import time
import pandas as pd
import numpy as np
from multiprocessing import Pool
import boto3

In [78]:
s3 = boto3.resource('s3')
client = boto3.client('s3')

In [79]:
edges_csv = os.listdir('/home/ec2-user/osm/edges/')
n_csvs = len(edges_csv)
edges_csv[0]
n_csvs

3110

In [80]:
def processCSVs(n):
    edges = pd.read_csv('/home/ec2-user/osm/edges/'+edges_csv[n], low_memory=False)
    # number of edges
    nedges = edges.geometry.size
    # get longitude/latitude dataFrame for edges
    df_edge_loc = pd.DataFrame()
    print('Creating exploded dataframe for '+edges_csv[n].replace('.csv',''))
    for i in range(0, nedges):
        edge_string = edges['geometry'][i][12:-1].split(",")
        tmp = []
        for j in range(0, len(edge_string)):
            edge_array = np.fromstring(edge_string[j], sep=' ')
            edge_array = np.flip(edge_array,axis=0)
            tmp.append(edge_array)
            tmp2 = np.concatenate(tmp)
            dftmp = pd.DataFrame(tmp2)
        df_edge_loc = pd.concat([df_edge_loc, dftmp], axis=1, ignore_index=True)
    path = '/home/ec2-user/osm/edges_exploded/'+edges_csv[n].replace('.csv','_exploded.csv')
    df_edge_loc.to_csv(path)
    del df_edge_loc
    # write CSV to s3
    s3.meta.client.upload_file(path, 'usa-edges-for-speed-mapping', 'edges_exploded/{}'.format(edges_csv[n].replace('.csv','_exploded.csv')))
    print('Saved '+path+' to S3.')    
    s3dir = client.list_objects(Bucket='usa-edges-for-speed-mapping',
                Prefix='edges_exploded/')
    # remove CSV from local EC2 instance.
    os.remove(path)
    s3dir = client.list_objects(Bucket='usa-edges-for-speed-mapping',
                Prefix='edges_exploded/')
    print((len(s3dir['Contents']) - 1)/len(edges_csv)*100, 'percent complete.')

    
    
    


In [ ]:
t0 = time.time();
if __name__ == '__main__':
    with Pool(processes=60) as p:
        p.map(processCSVs, range(0,n_csvs))
print(time.time() - t0)

Creating exploded dataframe for MS_IssaquenaCounty
Creating exploded dataframe for VA_FallsChurchcity
Creating exploded dataframe for ND_RenvilleCounty
Creating exploded dataframe for NC_JonesCounty
Creating exploded dataframe for MS_FranklinCounty
Creating exploded dataframe for IL_MoultrieCounty
Creating exploded dataframe for SD_AuroraCounty
Creating exploded dataframe for WV_TuckerCounty
Creating exploded dataframe for TX_GoliadCounty
Creating exploded dataframe for KS_JacksonCounty
Creating exploded dataframe for IA_DelawareCounty
Creating exploded dataframe for MS_ScottCounty
Creating exploded dataframe for ND_LaMoureCounty
Creating exploded dataframe for MS_LeakeCounty
Creating exploded dataframe for IA_UnionCounty
Creating exploded dataframe for OR_WashingtonCounty
Creating exploded dataframe for SD_McPhersonCounty
Creating exploded dataframe for MN_PipestoneCounty
Creating exploded dataframe for GA_GradyCounty
Creating exploded dataframe for GA_WorthCounty
Creating exploded da

In [13]:
# Test parallelization speed up of CSV processing...

# Serial Test:

t0 = time.time();
for n in range(0,2):
    processCSVs(n)
print('Serial process took ',time.time() - t0, ' seconds')

Creating exploded dataframe for WV_TuckerCounty
Saved /home/ec2-user/osm/edges_exploded/WV_TuckerCounty_exploded.csv
0.0  percent complete.
Creating exploded dataframe for KY_MarshallCounty
Saved /home/ec2-user/osm/edges_exploded/KY_MarshallCounty_exploded.csv
0.03215434083601286  percent complete.
Serial process took  124.81829786300659  seconds


In [3]:
os.cpu_count()

64